In [ ]:
!pip install -q transformers accelerate datasets


In [ ]:
!apt update
!apt install -y libmariadb-dev

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
38 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
!pip install mysql-connector-python sqlalchemy mariadb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/Horizontal-Labs/Argument-Mining.git

fatal: destination path 'Argument-Mining' already exists and is not an empty directory.


In [ ]:
# change direcotry
import sys
sys.path.append('/content/Argument-Mining')

In [ ]:
! pip install -U "huggingface_hub[cli]"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import random
import pandas as pd
import torch
import os
from huggingface_hub import login
login("your_HF_Token")


In [ ]:
MODEL_NAME =  "TinyLlama/TinyLlama-1.1B-Chat-v1.0" #"mistralai/Mistral-7B-Instruct-v0.1"#"tiiuae/falcon-rw-1b"#"google/flan-t5-large"# "google/flan-t5-base"#"mistralai/Mistral-7B-Instruct-v0.1"  # or "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Pipeline for generation
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


In [ ]:
from db.db import get_engine, get_session, get_db_session

# Get a session
session = get_session()

In [ ]:

try:
    from db.queries import get_test_data
    claims, premises, labels = get_test_data()
    data = [
        {"claim": c.text, "premise": p.text, "label": l}
        for c, p, l in zip(claims[:100], premises[:100], labels[:100])
    ]
except Exception as e:
    print("Failed to load test data from database. Falling back to mock data.")
    print("Error:", e)
    data = [
        {
            "claim": "Social media has a negative impact on mental health.",
            "premise": "Studies show increased anxiety and depression among teens using social platforms.",
            "label": "stance_pro"
        },
        {
            "claim": "School uniforms should be mandatory.",
            "premise": "Uniforms limit students' freedom of expression and individuality.",
            "label": "stance_con"
        }
    ] * 50



In [ ]:
TINYLLAMA_PROMPT = lambda claim, premise: f"""
Claim: {claim}
Evidence: {premise}
Does the evidence support or refute the claim? Respond with one word: "Supports" or "Refutes".
"""

A_FEW_SHOT_PROMPT = lambda claim, premise: f"""
You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Stance: Supports

Example 4:
Claim: Exercise is harmful to health.
Evidence: Numerous studies show that regular exercise improves cardiovascular health and longevity.
Stance: Refutes

Example 5:
Claim: Climate change is primarily caused by human activity.
Evidence: Recent studies show that over 90% of the increase in greenhouse gases is due to human activities like burning fossil fuels and deforestation.
Stance: Supports

Example 6:
Claim: Video games can improve cognitive skills.
Evidence: Research indicates that playing strategy games can improve memory, attention, and problem-solving skills.
Stance: Supports

Example 7:
Claim: Exercise has no effect on weight loss.
Evidence: Scientific research consistently shows that physical activity increases energy expenditure and supports weight management.
Stance: Refutes

Example 8:
Claim: Social media has no impact on mental health.
Evidence: Multiple studies have linked social media usage with increased rates of anxiety and depression in teenagers and young adults.
Stance: Refutes


Now analyze the following:

Claim: {claim}
Evidence: {premise}
Stance:"""

B_FEW_SHOT_PROMPT = lambda claim, premise: f"""
You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.

Example 1:
Claim: Climate change is primarily caused by human activity.
Evidence: Recent studies show that over 90% of the increase in greenhouse gases is due to human activities like burning fossil fuels and deforestation.
Stance: Supports

Example 2:
Claim: Video games can improve cognitive skills.
Evidence: Research indicates that playing strategy games can improve memory, attention, and problem-solving skills.
Stance: Supports

Example 3:
Claim: Exercise has no effect on weight loss.
Evidence: Scientific research consistently shows that physical activity increases energy expenditure and supports weight management.
Stance: Refutes

Example 4:
Claim: Social media has no impact on mental health.
Evidence: Multiple studies have linked social media usage with increased rates of anxiety and depression in teenagers and young adults.
Stance: Refutes

Example 5:
Claim: Electric cars are better for the environment than gasoline cars.
Evidence: Electric cars emit significantly fewer greenhouse gases compared to gasoline cars, even when accounting for electricity production.
Stance: Supports

Example 6:
Claim: The Earth is flat.
Evidence: Satellite imagery and circumnavigation of the globe confirm that the Earth is round.
Stance: Refutes

Example 7:
Claim: All processed foods are healthy.
Evidence: Nutrition experts warn that processed foods often contain added sugars, unhealthy fats, and preservatives.
Stance: Refutes


Example 8:
Claim: Exercising regularly improves mental health.
Evidence: Several clinical studies have found that consistent exercise helps reduce symptoms of depression and anxiety.
Stance: Supports

Example 9:
Claim: Vaccines are effective in preventing diseases.
Evidence: Data from the CDC shows that vaccination reduces the risk of infection by up to 95% in most cases.
Stance: Supports

Example 10:
Claim: Smoking is harmless to health.
Evidence: The World Health Organization states that smoking is a leading cause of cancer, respiratory diseases, and cardiovascular issues.
Stance: Refutes

Now, analyse the following and reply with only one word: either "Supports" or "Refutes" accordingly:
Claim: {claim}
Evidence: {premise}
Stance:"""




In [ ]:

import re

def normalize_output(output: str) -> str:
    output = output.strip().lower()

    # Find all stance lines
    stance_lines = re.findall(r"stance\s*[:\-]?\s*(supports|refutes)", output)

    if stance_lines:
        # Take the LAST one, since it's the final response
        return stance_lines[-1]

    # Fallback: try to get first matching standalone word
    for word in output.split():
        if "supports" in word:
            return "supports"
        elif "refutes" in word:
            return "refutes"

    return "unknown"



In [ ]:
reverse_label_map = {
    "supports": "stance_pro",
    "refutes": "stance_con"
}


In [ ]:
results = []
MODE = "few-shot"  # or "zero-shot"

for example in data:
    claim = example["claim"]
    premise = example["premise"]
    gold = example["label"]


    prompt = A_FEW_SHOT_PROMPT(claim, premise)



    # Truncate input if needed
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=10, do_sample=False, temperature=0.0)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    normalized_prediction = normalize_output(decoded)  # "Supports", "Refutes", "NotEnoughInfo"
    mapped_prediction = reverse_label_map.get(normalized_prediction, "unknown")  # → stance_pro, stance_con, unknown

    # DEBUG print
    first_token = decoded.lower().strip().split()[0].strip(".,:")
    print(f"🔍 Raw: {decoded} → Token: {first_token} → Label: {normalized_prediction}")

    results.append({
        "claim": claim,
        "premise": premise,
        "true_label": gold,
        "model_output": decoded.strip(),
        "normalized_prediction": normalized_prediction,  # Keep both
        "mapped_prediction": mapped_prediction,
        "correct": mapped_prediction == gold
    })


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔍 Raw: You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Stance: Supports

Example 4:
Claim: Exercise is harmful to health.
Evidence: Numerous studies show that regular exercise improves cardiovascular health and longevity.
Stance: Refutes

Example 5:  
Claim: Climate change is primarily caused by human activity.  
Evidence: Recent studies show that over 90% of the increase in greenhouse gases is due to 

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔍 Raw: You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Stance: Supports

Example 4:
Claim: Exercise is harmful to health.
Evidence: Numerous studies show that regular exercise improves cardiovascular health and longevity.
Stance: Refutes

Example 5:  
Claim: Climate change is primarily caused by human activity.  
Evidence: Recent studies show that over 90% of the increase in greenhouse gases is due to 

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

🔍 Raw: You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Stance: Supports

Example 4:
Claim: Exercise is harmful to health.
Evidence: Numerous studies show that regular exercise improves cardiovascular health and longevity.
Stance: Refutes

Example 5:  
Claim: Climate change is primarily caused by human activity.  
Evidence: Recent studies show that over 90% of the increase in greenhouse gases is due to 

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔍 Raw: You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Stance: Supports

Example 4:
Claim: Exercise is harmful to health.
Evidence: Numerous studies show that regular exercise improves cardiovascular health and longevity.
Stance: Refutes

Example 5:  
Claim: Climate change is primarily caused by human activity.  
Evidence: Recent studies show that over 90% of the increase in greenhouse gases is due to 

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔍 Raw: You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Stance: Supports

Example 4:
Claim: Exercise is harmful to health.
Evidence: Numerous studies show that regular exercise improves cardiovascular health and longevity.
Stance: Refutes

Example 5:  
Claim: Climate change is primarily caused by human activity.  
Evidence: Recent studies show that over 90% of the increase in greenhouse gases is due to 

In [ ]:
print(f"🔍 Normalized: {normalized_prediction} → Mapped label: {mapped_prediction}")


🔍 Normalized: supports → Mapped label: stance_pro


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

results_df = pd.DataFrame(results)

'''# Optional: drop "unknown" (NotEnoughInfo) predictions
filtered_df = results_df[results_df["mapped_prediction"] != "unknown"]'''
y_true = results_df["true_label"]
y_pred = results_df["mapped_prediction"]

print(f"\nFinal Accuracy ({MODE} using {MODEL_NAME}): {accuracy_score(y_true, y_pred):.2%}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred, labels=["stance_pro", "stance_con"]))
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred, labels=["stance_pro", "stance_con"]))

acc = results_df["correct"].mean()
print(acc)



Final Accuracy (few-shot using TinyLlama/TinyLlama-1.1B-Chat-v1.0): 58.00%

Classification Report:
              precision    recall  f1-score   support

  stance_pro       0.58      0.53      0.55        49
  stance_con       0.58      0.63      0.60        51

    accuracy                           0.58       100
   macro avg       0.58      0.58      0.58       100
weighted avg       0.58      0.58      0.58       100


Confusion Matrix:
[[26 23]
 [19 32]]
0.58


In [ ]:
import pandas as pd
import os

# Assuming 'results' is your final list of dictionaries
# Convert directly to DataFrame
s_df = pd.DataFrame(results)

# Columns to keep
columns_to_export = ["claim", "premise", "true_label","normalized_prediction", "mapped_prediction", "model_output", "correct"]

# Check if file already exists
s_file = "stance_predictions_summary.csv"
if os.path.exists(s_file):
    # Append only the selected columns to the existing CSV
    s_df[columns_to_export].to_csv(s_file, mode='a', header=False, index=False)
else:
    # Create new CSV with the header
    s_df[columns_to_export].to_csv(s_file, index=False)

print("✅ CSV with selected columns saved as 'stance_predictions_summary.csv'!")


✅ CSV with selected columns saved as 'stance_predictions_summary.csv'!


In [ ]:
from datetime import datetime
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

if "experiment_log" not in globals():
    experiment_log = pd.DataFrame(columns=[
        "timestamp", "model", "mode", "accuracy",
        "precision_pro", "recall_pro", "f1_pro",
        "precision_con", "recall_con", "f1_con"
    ])



report = classification_report(y_true, y_pred, labels=["stance_pro", "stance_con"], output_dict=True)

summary = {
    "timestamp": datetime.now().isoformat(timespec='seconds'),
    "model": MODEL_NAME,
    "mode": MODE,
    "accuracy": accuracy_score(y_true, y_pred),
    "precision_pro": precision_score(y_true, y_pred, pos_label="stance_pro", zero_division=0),
    "recall_pro": recall_score(y_true, y_pred, pos_label="stance_pro", zero_division=0),
    "f1_pro": f1_score(y_true, y_pred, pos_label="stance_pro", zero_division=0),
    "precision_con": precision_score(y_true, y_pred, pos_label="stance_con", zero_division=0),
    "recall_con": recall_score(y_true, y_pred, pos_label="stance_con", zero_division=0),
    "f1_con": f1_score(y_true, y_pred, pos_label="stance_con", zero_division=0),
}

experiment_log = pd.DataFrame(columns=summary.keys())
# Append and print updated table
experiment_log = pd.concat([experiment_log, pd.DataFrame([summary])], ignore_index=True)
print("📊 Summary of All Experiments So Far:")
display(experiment_log)


📊 Summary of All Experiments So Far:


<ipython-input-66-4ee94a84f8af>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  experiment_log = pd.concat([experiment_log, pd.DataFrame([summary])], ignore_index=True)


,timestamp,model,mode,accuracy,precision_pro,recall_pro,f1_pro,precision_con,recall_con,f1_con
0,2025-06-02T21:18:28,TinyLlama/TinyLlama-1.1B-Chat-v1.0,few-shot,0.58,0.577778,0.530612,0.553191,0.581818,0.627451,0.603774


In [ ]:
summary_file = "stance_eval_summary.csv"
summary_df = pd.DataFrame([summary])

# Append to CSV (create if it doesn't exist)
if os.path.exists(summary_file):
    summary_df.to_csv(summary_file, mode='a', header=False, index=False)
else:
    summary_df.to_csv(summary_file, index=False)


In [ ]:
if os.path.exists(summary_file):
    print("\n📊 Summary of All Experiments So Far:")
    display(pd.read_csv(summary_file))
